In [129]:
import numpy as np 
import pandas as pd 
from keras.datasets import imdb
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D, Convolution1D, BatchNormalization, Dropout, Flatten, SpatialDropout1D, Input
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import datetime
import time
import backtrader as bt
%matplotlib inline


In [130]:

def extract_dataset():
    data = pd.read_csv('standardchartered.csv')
    data = data.drop(['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
    # Keeping only the neccessary columns
    print('Extracting dataset...')
    df = data.set_index('Date')
    perChange = (df['Close'].shift(-1) / df['Close']) - 1
    y = perChange > 0
    y = y.astype(int)
    #print(data)
    return y, df

In [131]:
y, df = extract_dataset()
Xtrain, Xtest, ytrain, ytest = train_test_split(df, y, test_size=0.2, shuffle=False)
X_train = np.asarray(Xtrain)
X_test = np.asarray(Xtest)

y_train = np.asarray(ytrain)
y_test = np.asarray(ytest)


Extracting dataset...


In [132]:
def model():
    inp = Input(shape=(None, 5))
        
    x = LSTM(256,return_sequences=True, activation='relu')(inp)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = LSTM(256, return_sequences=True, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    x = LSTM(256, activation='softmax')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    
    x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp, x)
    
    model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train.reshape(-1, 1, 5), y_train, epochs=20)
    return model
model = model()

Epoch 1/20
400/400 [==============================] - 8s 21ms/step - loss: 0.6948 - acc: 0.4625
Epoch 2/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6928 - acc: 0.5250
Epoch 3/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6882 - acc: 0.5475
Epoch 4/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6941 - acc: 0.5175
Epoch 5/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6903 - acc: 0.5200
Epoch 6/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6896 - acc: 0.5400
Epoch 7/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6945 - acc: 0.5150
Epoch 8/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6883 - acc: 0.5350
Epoch 9/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6943 - acc: 0.4950
Epoch 10/20
400/400 [==============================] - 1s 2ms/step - loss: 0.6930 - acc: 0.5375
Epoch 11/20
400/400 [===========================

In [133]:
#
probdata=[]
data=[]
data = pd.read_csv('standardchartered.csv')
#data = data.drop(['Date', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
data = data.drop(['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'], axis=1)
data = data.set_index('Date')
for i in range (0, data.shape[0]):
    x=np.array(pd.DataFrame(data.iloc[[i]]))
    y  = model.predict(x.reshape(-1, 1, 5))
    #print("Probability: "+ str(y[0]))
    probdata.append(y[0])

output = pd.DataFrame(probdata, columns=["Probability"])
#print(output)


In [134]:
output['Date'] = data.index

In [135]:
output = output.set_index('Date')

In [136]:
output.head()

,Probability
Date,
3/24/2011,0.483786
3/25/2011,0.498382
3/27/2011,0.434765
3/28/2011,0.426493
3/29/2011,0.488677


In [137]:
data.head()

,Open,High,Low,Close,Volume
Date,,,,,
3/24/2011,1785,1760,1700,1700,1140
3/25/2011,1700,1734,1676,1734,1252
3/27/2011,1734,1714,1690,1690,157
3/28/2011,1690,1680,1627,1627,270
3/29/2011,1627,1625,1568,1570,1150


In [138]:
finalData = data.join(output)
finalData

,Open,High,Low,Close,Volume,Probability
Date,,,,,,
1/1/2012,1340,1370,1340,1355,1155,0.515502
1/1/2013,2115,2151,2095,2105,1609,0.521568
1/10/2012,1384,1375,1300,1352,1520,0.531551
1/10/2013,2050,2049,2000,2049,2181,0.502094
1/11/2012,1352,1399,1305,1399,904,0.493403
1/12/2012,1399,1410,1340,1410,437,0.434012
1/13/2013,2049,2065,2020,2065,781,0.492280
1/15/2013,2065,2053,2004,2053,1500,0.513257
1/16/2012,1410,1387,1341,1370,960,0.487411


In [139]:
finalData.to_csv('analyzedData.csv', sep='\t', encoding='utf-8')